# 🧩 Modular RAG Implementation

## Complete Modular Architecture for AI Systems

This notebook demonstrates a **Modular RAG system** with:
- 🔍 Intelligent Query Processing
- 🔎 Multi-Strategy Retrieval 
- 🤖 Adaptive Generation
- 💾 Memory Management
- 🚦 Smart Routing

### Key Benefits of Modular Architecture
- **Flexibility**: Easy to swap and customize components
- **Scalability**: Add new modules without changing core system
- **Maintainability**: Independent testing and updates
- **Extensibility**: Domain-specific specializations

In [2]:
# Install required packages
!pip install sentence-transformers faiss-cpu google-generativeai rank-bm25 transformers scikit-learn numpy python-dotenv

You should consider upgrading via the '/home/mohdasimkhan/.pyenv/versions/3.10.2/envs/rags/bin/python -m pip install --upgrade pip' command.


In [3]:
# Import libraries
import numpy as np
import json
import re
import os
import time
import uuid
from typing import List, Dict, Tuple, Optional, Any
from dataclasses import dataclass, field
from abc import ABC, abstractmethod
from enum import Enum
from datetime import datetime, timedelta

from sentence_transformers import SentenceTransformer
import faiss
import google.generativeai as genai
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction.text import TfidfVectorizer
from dotenv import load_dotenv

load_dotenv()
print("📚 Libraries imported successfully!")

/home/mohdasimkhan/.pyenv/versions/3.10.2/envs/rags/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📚 Libraries imported successfully!


## 🏗️ Core Data Structures

Define the foundational structures for our modular system:

In [4]:
# Enums for system configuration
class QueryType(Enum):
    FACTUAL = "factual"
    COMPARATIVE = "comparative"
    PROCEDURAL = "procedural"
    ANALYTICAL = "analytical"
    CONVERSATIONAL = "conversational"

class Domain(Enum):
    TECHNOLOGY = "technology"
    SCIENCE = "science"
    BUSINESS = "business"
    HEALTHCARE = "healthcare"
    GENERAL = "general"

class RetrievalStrategy(Enum):
    SEMANTIC = "semantic"
    KEYWORD = "keyword"
    HYBRID = "hybrid"

# Core data structures
@dataclass
class Document:
    id: str
    title: str
    content: str
    domain: Domain
    keywords: List[str] = field(default_factory=list)
    embedding: Optional[np.ndarray] = None

@dataclass
class Query:
    id: str
    text: str
    query_type: QueryType
    domain: Domain
    user_id: Optional[str] = None
    session_id: Optional[str] = None

@dataclass
class RetrievalResult:
    document: Document
    score: float
    rank: int
    strategy_used: RetrievalStrategy

@dataclass
class ModularResponse:
    query: Query
    retrieved_documents: List[RetrievalResult]
    generated_answer: str
    confidence_score: float
    processing_pipeline: List[str]
    processing_time: float

print("🏗️ Data structures defined!")

🏗️ Data structures defined!


## 📚 Multi-Domain Knowledge Base

Create knowledge base covering multiple domains:

In [5]:
# Multi-domain knowledge base
knowledge_base = [
    {
        "id": "tech_001",
        "title": "Cloud Computing Fundamentals",
        "domain": "technology",
        "content": "Cloud computing delivers computing services including servers, storage, databases, networking, software, analytics, and intelligence over the internet. Main service models include Infrastructure as a Service (IaaS), Platform as a Service (PaaS), and Software as a Service (SaaS). Major providers include AWS, Microsoft Azure, and Google Cloud Platform. Benefits include cost reduction, scalability, reliability, and global reach.",
        "keywords": ["cloud", "computing", "AWS", "Azure", "IaaS", "PaaS", "SaaS"]
    },
    {
        "id": "tech_002",
        "title": "Artificial Intelligence and Machine Learning",
        "domain": "technology",
        "content": "Artificial Intelligence simulates human intelligence in machines programmed to think and learn. Machine Learning is a subset of AI that enables systems to learn from experience without explicit programming. Deep Learning uses neural networks with multiple layers. Key applications include computer vision, natural language processing, recommendation systems, and autonomous vehicles.",
        "keywords": ["AI", "machine learning", "deep learning", "neural networks", "computer vision"]
    },
    {
        "id": "sci_001",
        "title": "Quantum Computing Principles",
        "domain": "science",
        "content": "Quantum computing leverages quantum mechanical phenomena like superposition and entanglement to process information differently than classical computers. Quantum bits (qubits) can exist in multiple states simultaneously. Key principles include superposition, entanglement, and quantum interference. Applications include cryptography, optimization, drug discovery, and financial modeling.",
        "keywords": ["quantum", "qubits", "superposition", "entanglement", "cryptography"]
    },
    {
        "id": "health_001",
        "title": "Telemedicine and Digital Health",
        "domain": "healthcare",
        "content": "Telemedicine uses telecommunications technology to provide clinical health care at a distance. Key modalities include video consultations, remote patient monitoring, mobile health applications, and digital therapeutics. Benefits include improved access to care, reduced costs, convenience, and ability to serve remote populations. COVID-19 accelerated telemedicine adoption significantly.",
        "keywords": ["telemedicine", "digital health", "remote monitoring", "mobile health"]
    },
    {
        "id": "bus_001",
        "title": "Digital Transformation Strategies",
        "domain": "business",
        "content": "Digital transformation integrates digital technology into all business areas, fundamentally changing operations and value delivery. Key components include process automation, data analytics, customer experience enhancement, and business model innovation. Success factors include leadership commitment, cultural change management, employee training, and customer-centric focus.",
        "keywords": ["digital transformation", "automation", "analytics", "customer experience"]
    }
]

print(f"📚 Knowledge base created with {len(knowledge_base)} documents")
print(f"🌍 Domains: {set(doc['domain'] for doc in knowledge_base)}")

📚 Knowledge base created with 5 documents
🌍 Domains: {'healthcare', 'science', 'business', 'technology'}


## 🧩 Module Base Classes

Define abstract base classes for modular architecture:

In [6]:
# Base module class
class BaseModule(ABC):
    def __init__(self, name: str):
        self.name = name
        self.call_count = 0
        self.created_at = datetime.now()
    
    @abstractmethod
    def process(self, input_data: Any) -> Any:
        pass
    
    def update_stats(self):
        self.call_count += 1
    
    def get_info(self):
        return {
            'name': self.name,
            'calls': self.call_count,
            'created': self.created_at
        }

print("🧩 Base module class defined!")

🧩 Base module class defined!


## 🔍 Query Processing Module

Intelligent query understanding and classification:

In [7]:
class QueryModule(BaseModule):
    def __init__(self):
        super().__init__("QueryModule")
        
        # Query type patterns
        self.type_patterns = {
            QueryType.FACTUAL: [r"what is", r"define", r"explain"],
            QueryType.COMPARATIVE: [r"compare", r"difference", r"versus", r"vs"],
            QueryType.PROCEDURAL: [r"how to", r"steps", r"process"],
            QueryType.ANALYTICAL: [r"analyze", r"why", r"impact"],
            QueryType.CONVERSATIONAL: [r"hello", r"hi", r"thanks"]
        }
        
        # Domain keywords
        self.domain_keywords = {
            Domain.TECHNOLOGY: ["AI", "cloud", "software", "computing", "algorithm"],
            Domain.SCIENCE: ["quantum", "research", "experiment", "theory"],
            Domain.HEALTHCARE: ["medicine", "health", "patient", "treatment"],
            Domain.BUSINESS: ["business", "strategy", "management", "revenue"]
        }
    
    def process(self, query_text: str) -> Query:
        self.update_stats()
        
        query_id = str(uuid.uuid4())[:8]
        query_type = self._detect_type(query_text)
        domain = self._detect_domain(query_text)
        
        return Query(
            id=query_id,
            text=query_text,
            query_type=query_type,
            domain=domain
        )
    
    def _detect_type(self, text: str) -> QueryType:
        text_lower = text.lower()
        
        for query_type, patterns in self.type_patterns.items():
            for pattern in patterns:
                if re.search(pattern, text_lower):
                    return query_type
        
        return QueryType.FACTUAL
    
    def _detect_domain(self, text: str) -> Domain:
        text_lower = text.lower()
        domain_scores = {}
        
        for domain, keywords in self.domain_keywords.items():
            score = sum(1 for keyword in keywords if keyword.lower() in text_lower)
            domain_scores[domain] = score
        
        if max(domain_scores.values()) > 0:
            return max(domain_scores, key=domain_scores.get)
        return Domain.GENERAL

# Test query module
query_module = QueryModule()
test_query = query_module.process("What is cloud computing?")
print(f"🔍 Test query processed:")
print(f"   Text: {test_query.text}")
print(f"   Type: {test_query.query_type.value}")
print(f"   Domain: {test_query.domain.value}")
print("✅ Query Module ready!")

🔍 Test query processed:
   Text: What is cloud computing?
   Type: factual
   Domain: technology
✅ Query Module ready!


## 🔎 Multi-Strategy Retrieval Module

Flexible retrieval with multiple strategies:

In [8]:
class RetrievalModule(BaseModule):
    def __init__(self):
        super().__init__("RetrievalModule")
        
        # Initialize components
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.documents = []
        self.semantic_index = None
        self.bm25_index = None
        
        print("🔎 Retrieval module initialized")
    
    def index_documents(self, documents: List[Dict]):
        print(f"📚 Indexing {len(documents)} documents...")
        
        # Convert to Document objects
        self.documents = [
            Document(
                id=doc['id'],
                title=doc['title'],
                content=doc['content'],
                domain=Domain(doc['domain']),
                keywords=doc.get('keywords', [])
            )
            for doc in documents
        ]
        
        # Build semantic index
        self._build_semantic_index()
        
        # Build keyword index
        self._build_keyword_index()
        
        print("✅ Documents indexed successfully!")
    
    def _build_semantic_index(self):
        doc_texts = [f"{doc.title} {doc.content}" for doc in self.documents]
        embeddings = self.embedding_model.encode(doc_texts)
        
        # Store embeddings
        for doc, embedding in zip(self.documents, embeddings):
            doc.embedding = embedding
        
        # Create FAISS index
        dimension = embeddings.shape[1]
        self.semantic_index = faiss.IndexFlatIP(dimension)
        faiss.normalize_L2(embeddings)
        self.semantic_index.add(embeddings.astype('float32'))
    
    def _build_keyword_index(self):
        doc_texts = [f"{doc.title} {doc.content}" for doc in self.documents]
        tokenized_docs = [text.lower().split() for text in doc_texts]
        self.bm25_index = BM25Okapi(tokenized_docs)
    
    def process(self, query: Query, strategy: RetrievalStrategy = RetrievalStrategy.HYBRID, top_k: int = 5) -> List[RetrievalResult]:
        self.update_stats()
        
        if strategy == RetrievalStrategy.SEMANTIC:
            return self._semantic_search(query, top_k)
        elif strategy == RetrievalStrategy.KEYWORD:
            return self._keyword_search(query, top_k)
        else:  # HYBRID
            return self._hybrid_search(query, top_k)
    
    def _semantic_search(self, query: Query, top_k: int) -> List[RetrievalResult]:
        query_embedding = self.embedding_model.encode([query.text])
        faiss.normalize_L2(query_embedding)
        
        scores, indices = self.semantic_index.search(query_embedding.astype('float32'), top_k)
        
        results = []
        for i, (score, idx) in enumerate(zip(scores[0], indices[0])):
            results.append(RetrievalResult(
                document=self.documents[idx],
                score=float(score),
                rank=i + 1,
                strategy_used=RetrievalStrategy.SEMANTIC
            ))
        return results
    
    def _keyword_search(self, query: Query, top_k: int) -> List[RetrievalResult]:
        query_tokens = query.text.lower().split()
        scores = self.bm25_index.get_scores(query_tokens)
        top_indices = np.argsort(scores)[::-1][:top_k]
        
        results = []
        for i, idx in enumerate(top_indices):
            results.append(RetrievalResult(
                document=self.documents[idx],
                score=float(scores[idx]),
                rank=i + 1,
                strategy_used=RetrievalStrategy.KEYWORD
            ))
        return results
    
    def _hybrid_search(self, query: Query, top_k: int) -> List[RetrievalResult]:
        # Get results from both strategies
        semantic_results = self._semantic_search(query, top_k * 2)
        keyword_results = self._keyword_search(query, top_k * 2)
        
        # Normalize scores
        self._normalize_scores(semantic_results)
        self._normalize_scores(keyword_results)
        
        # Combine with weights
        combined_scores = {}
        semantic_weight = 0.7
        keyword_weight = 0.3
        
        for result in semantic_results:
            doc_id = result.document.id
            combined_scores[doc_id] = {
                'document': result.document,
                'semantic_score': result.score,
                'keyword_score': 0.0
            }
        
        for result in keyword_results:
            doc_id = result.document.id
            if doc_id in combined_scores:
                combined_scores[doc_id]['keyword_score'] = result.score
            else:
                combined_scores[doc_id] = {
                    'document': result.document,
                    'semantic_score': 0.0,
                    'keyword_score': result.score
                }
        
        # Calculate final scores
        final_results = []
        for doc_id, scores in combined_scores.items():
            final_score = (semantic_weight * scores['semantic_score'] + 
                          keyword_weight * scores['keyword_score'])
            
            final_results.append(RetrievalResult(
                document=scores['document'],
                score=final_score,
                rank=0,
                strategy_used=RetrievalStrategy.HYBRID
            ))
        
        # Sort and assign ranks
        final_results.sort(key=lambda x: x.score, reverse=True)
        for i, result in enumerate(final_results[:top_k]):
            result.rank = i + 1
        
        return final_results[:top_k]
    
    def _normalize_scores(self, results: List[RetrievalResult]):
        if not results:
            return
        
        scores = [result.score for result in results]
        min_score, max_score = min(scores), max(scores)
        
        if max_score > min_score:
            for result in results:
                result.score = (result.score - min_score) / (max_score - min_score)

# Initialize retrieval module
retrieval_module = RetrievalModule()
retrieval_module.index_documents(knowledge_base)

print("✅ Retrieval Module ready!")

🔎 Retrieval module initialized
📚 Indexing 5 documents...
✅ Documents indexed successfully!
✅ Retrieval Module ready!


## 🤖 Adaptive Generation Module

Context-aware answer generation:

In [9]:
class GenerationModule(BaseModule):
    def __init__(self):
        super().__init__("GenerationModule")
        
        # Try to initialize Gemini
        api_key = os.getenv('GEMINI_API_KEY')
        if api_key:
            try:
                genai.configure(api_key=api_key)
                self.model = genai.GenerativeModel('gemini-1.5-flash')
                self.has_llm = True
                print("🤖 Gemini API configured")
            except Exception as e:
                print(f"⚠️ Gemini error: {e}")
                self.has_llm = False
        else:
            print("⚠️ No Gemini API key. Using template generation.")
            self.has_llm = False
    
    def process(self, query: Query, retrieved_docs: List[RetrievalResult]) -> Tuple[str, float]:
        self.update_stats()
        
        if self.has_llm:
            return self._generate_with_llm(query, retrieved_docs)
        else:
            return self._generate_with_template(query, retrieved_docs)
    
    def _generate_with_llm(self, query: Query, retrieved_docs: List[RetrievalResult]) -> Tuple[str, float]:
        # Prepare context
        context_parts = []
        for result in retrieved_docs:
            doc = result.document
            context_parts.append(f"Title: {doc.title}\nContent: {doc.content}")
        
        context = "\n\n".join(context_parts)
        
        # Create prompt based on query type
        if query.query_type == QueryType.COMPARATIVE:
            instruction = "Compare and contrast the concepts mentioned in the context."
        elif query.query_type == QueryType.PROCEDURAL:
            instruction = "Provide step-by-step guidance based on the context."
        elif query.query_type == QueryType.ANALYTICAL:
            instruction = "Provide a detailed analysis based on the context."
        else:
            instruction = "Provide a clear, informative answer based on the context."
        
        prompt = f"""You are an expert assistant specializing in {query.domain.value}. 
Use the provided context to answer the user's question.

Context:
{context}

Question: {query.text}
Query Type: {query.query_type.value}

{instruction}

Answer:"""
        
        try:
            response = self.model.generate_content(prompt)
            return response.text, 0.9
        except Exception as e:
            return f"Error generating response: {str(e)}", 0.0
    
    def _generate_with_template(self, query: Query, retrieved_docs: List[RetrievalResult]) -> Tuple[str, float]:
        if not retrieved_docs:
            return "I couldn't find relevant information to answer your question.", 0.1
        
        # Create template-based response
        if query.query_type == QueryType.COMPARATIVE:
            answer = f"Comparing the concepts related to '{query.text}':\n\n"
            for i, result in enumerate(retrieved_docs[:2], 1):
                doc = result.document
                answer += f"{i}. **{doc.title}**: {doc.content[:200]}...\n\n"
        else:
            top_doc = retrieved_docs[0].document
            answer = f"Based on the available information about {query.domain.value}, "
            answer += f"here's what I found about '{query.text}':\n\n"
            answer += f"**{top_doc.title}**: {top_doc.content}"
            
            if len(retrieved_docs) > 1:
                answer += f"\n\nAdditional relevant information from {len(retrieved_docs)-1} other sources is also available."
        
        confidence = min(0.8, len(retrieved_docs) / 3.0)
        return answer, confidence

# Initialize generation module
generation_module = GenerationModule()
print("✅ Generation Module ready!")

🤖 Gemini API configured
✅ Generation Module ready!


## 💾 Memory Module

Session and user context management:

In [10]:
class MemoryModule(BaseModule):
    def __init__(self):
        super().__init__("MemoryModule")
        self.user_sessions = {}
        self.query_history = []
        self.max_history = 100
    
    def process(self, query: Query, response: ModularResponse):
        self.update_stats()
        
        # Store interaction
        interaction = {
            'timestamp': datetime.now(),
            'query': query.text,
            'query_type': query.query_type.value,
            'domain': query.domain.value,
            'confidence': response.confidence_score,
            'processing_time': response.processing_time
        }
        
        self.query_history.append(interaction)
        
        # Limit history size
        if len(self.query_history) > self.max_history:
            self.query_history = self.query_history[-self.max_history:]
        
        # Update user session if available
        if query.user_id:
            if query.user_id not in self.user_sessions:
                self.user_sessions[query.user_id] = {
                    'queries': [],
                    'preferred_domains': {},
                    'last_active': datetime.now()
                }
            
            session = self.user_sessions[query.user_id]
            session['queries'].append(interaction)
            session['last_active'] = datetime.now()
            
            # Update domain preferences
            domain = query.domain.value
            if domain not in session['preferred_domains']:
                session['preferred_domains'][domain] = 0
            session['preferred_domains'][domain] += 1
    
    def get_user_context(self, user_id: str) -> Dict:
        if user_id in self.user_sessions:
            return self.user_sessions[user_id]
        return {}
    
    def get_statistics(self) -> Dict:
        return {
            'total_interactions': len(self.query_history),
            'active_users': len(self.user_sessions),
            'avg_confidence': np.mean([h['confidence'] for h in self.query_history]) if self.query_history else 0
        }

# Initialize memory module
memory_module = MemoryModule()
print("✅ Memory Module ready!")

✅ Memory Module ready!


## 🚦 Router Module

Intelligent routing and pipeline orchestration:

In [11]:
class RouterModule(BaseModule):
    def __init__(self):
        super().__init__("RouterModule")
        
        # Routing strategies based on query complexity
        self.routing_strategies = {
            'simple': {
                'retrieval_strategy': RetrievalStrategy.SEMANTIC,
                'max_docs': 3,
                'pipeline': ['query', 'retrieval', 'generation']
            },
            'medium': {
                'retrieval_strategy': RetrievalStrategy.HYBRID,
                'max_docs': 5,
                'pipeline': ['query', 'retrieval', 'generation', 'memory']
            },
            'complex': {
                'retrieval_strategy': RetrievalStrategy.HYBRID,
                'max_docs': 7,
                'pipeline': ['query', 'retrieval', 'generation', 'memory']
            }
        }
    
    def process(self, query: Query) -> Dict:
        self.update_stats()
        
        # Determine complexity
        complexity = self._assess_complexity(query)
        
        # Get routing strategy
        strategy = self.routing_strategies[complexity]
        
        return {
            'complexity': complexity,
            'retrieval_strategy': strategy['retrieval_strategy'],
            'max_docs': strategy['max_docs'],
            'pipeline': strategy['pipeline'],
            'estimated_time': self._estimate_time(complexity)
        }
    
    def _assess_complexity(self, query: Query) -> str:
        complexity_score = 0
        
        # Length-based complexity
        word_count = len(query.text.split())
        if word_count > 15:
            complexity_score += 2
        elif word_count > 8:
            complexity_score += 1
        
        # Query type complexity
        if query.query_type in [QueryType.ANALYTICAL, QueryType.COMPARATIVE]:
            complexity_score += 2
        elif query.query_type == QueryType.PROCEDURAL:
            complexity_score += 1
        
        # Domain complexity
        if query.domain in [Domain.SCIENCE, Domain.HEALTHCARE]:
            complexity_score += 1
        
        # Classify
        if complexity_score <= 1:
            return 'simple'
        elif complexity_score <= 3:
            return 'medium'
        else:
            return 'complex'
    
    def _estimate_time(self, complexity: str) -> float:
        time_estimates = {
            'simple': 1.0,
            'medium': 2.5,
            'complex': 4.0
        }
        return time_estimates.get(complexity, 2.0)

# Initialize router module
router_module = RouterModule()
print("✅ Router Module ready!")

✅ Router Module ready!


## 🧩 Complete Modular RAG System

Integrate all modules into the complete system:

In [12]:
class ModularRAGSystem:
    def __init__(self):
        print("🧩 Initializing Modular RAG System...")
        
        # Initialize all modules
        self.modules = {
            'query': query_module,
            'retrieval': retrieval_module,
            'generation': generation_module,
            'memory': memory_module,
            'router': router_module
        }
        
        # System metrics
        self.total_queries = 0
        self.successful_queries = 0
        self.avg_processing_time = 0.0
        
        print("✅ Modular RAG System initialized!")
        print(f"🔧 Active modules: {list(self.modules.keys())}")
    
    def process_query(self, query_text: str, user_id: str = None) -> ModularResponse:
        start_time = time.time()
        
        try:
            print(f"\n🔍 Processing: '{query_text}'")
            print("=" * 50)
            
            # Step 1: Query processing
            print("📝 Step 1: Query processing...")
            query = self.modules['query'].process(query_text)
            query.user_id = user_id
            
            print(f"   Type: {query.query_type.value}, Domain: {query.domain.value}")
            
            # Step 2: Routing
            print("🚦 Step 2: Intelligent routing...")
            routing_plan = self.modules['router'].process(query)
            
            print(f"   Complexity: {routing_plan['complexity']}")
            print(f"   Strategy: {routing_plan['retrieval_strategy'].value}")
            print(f"   Est. time: {routing_plan['estimated_time']:.1f}s")
            
            # Step 3: Retrieval
            print("🔍 Step 3: Document retrieval...")
            retrieved_docs = self.modules['retrieval'].process(
                query, 
                routing_plan['retrieval_strategy'], 
                routing_plan['max_docs']
            )
            
            print(f"   Retrieved {len(retrieved_docs)} documents")
            for i, doc in enumerate(retrieved_docs[:3], 1):
                print(f"   {i}. {doc.document.title} (Score: {doc.score:.3f})")
            
            # Step 4: Generation
            print("🤖 Step 4: Answer generation...")
            generated_answer, confidence = self.modules['generation'].process(query, retrieved_docs)
            
            print(f"   Generated answer (Confidence: {confidence:.2f})")
            
            # Create response
            processing_time = time.time() - start_time
            
            response = ModularResponse(
                query=query,
                retrieved_documents=retrieved_docs,
                generated_answer=generated_answer,
                confidence_score=confidence,
                processing_pipeline=routing_plan['pipeline'],
                processing_time=processing_time
            )
            
            # Step 5: Memory update
            print("💾 Step 5: Memory update...")
            self.modules['memory'].process(query, response)
            
            # Update system metrics
            self._update_metrics(response)
            
            print(f"\n✅ Query processed in {processing_time:.2f}s")
            return response
            
        except Exception as e:
            print(f"❌ Error: {str(e)}")
            # Return error response
            error_query = Query("error", query_text, QueryType.FACTUAL, Domain.GENERAL, user_id)
            return ModularResponse(
                query=error_query,
                retrieved_documents=[],
                generated_answer=f"Sorry, I encountered an error: {str(e)}",
                confidence_score=0.0,
                processing_pipeline=['error'],
                processing_time=time.time() - start_time
            )
    
    def _update_metrics(self, response: ModularResponse):
        self.total_queries += 1
        if response.confidence_score > 0.5:
            self.successful_queries += 1
        
        # Update running average
        self.avg_processing_time = ((self.avg_processing_time * (self.total_queries - 1)) + 
                                   response.processing_time) / self.total_queries
    
    def get_system_status(self) -> Dict:
        module_stats = {name: module.get_info() for name, module in self.modules.items()}
        
        return {
            'total_queries': self.total_queries,
            'successful_queries': self.successful_queries,
            'success_rate': (self.successful_queries / max(self.total_queries, 1)) * 100,
            'avg_processing_time': self.avg_processing_time,
            'module_stats': module_stats,
            'memory_stats': self.modules['memory'].get_statistics()
        }

# Initialize complete system
modular_rag = ModularRAGSystem()
print("\n🚀 Complete Modular RAG System ready!")

🧩 Initializing Modular RAG System...
✅ Modular RAG System initialized!
🔧 Active modules: ['query', 'retrieval', 'generation', 'memory', 'router']

🚀 Complete Modular RAG System ready!


## 🧪 Comprehensive Testing

Test the system with different query types and complexities:

In [13]:
def run_test_suite():
    print("\n" + "🧪" * 20 + " MODULAR RAG TEST SUITE " + "🧪" * 20)
    
    test_cases = [
        {
            "name": "Simple Factual Query",
            "query": "What is cloud computing?",
            "expected_complexity": "simple",
            "user_id": "test_user_1"
        },
        {
            "name": "Comparative Query",
            "query": "Compare artificial intelligence and machine learning",
            "expected_complexity": "medium",
            "user_id": "test_user_2"
        },
        {
            "name": "Complex Analytical Query",
            "query": "Analyze the impact of quantum computing on digital transformation strategies in healthcare",
            "expected_complexity": "complex",
            "user_id": "test_user_3"
        },
        {
            "name": "Procedural Query",
            "query": "How to implement telemedicine in healthcare systems?",
            "expected_complexity": "medium",
            "user_id": "test_user_4"
        }
    ]
    
    results = []
    total_start_time = time.time()
    
    for i, test_case in enumerate(test_cases, 1):
        print(f"\n{'=' * 60}")
        print(f"🔬 TEST CASE {i}: {test_case['name']}")
        print(f"❓ Query: '{test_case['query']}'")
        print("=" * 60)
        
        try:
            response = modular_rag.process_query(
                test_case['query'], 
                test_case['user_id']
            )
            
            # Display results
            print(f"\n🎯 **FINAL ANSWER:**")
            print(f"   {response.generated_answer}")
            
            print(f"\n📊 **METRICS:**")
            print(f"   • Processing Time: {response.processing_time:.2f}s")
            print(f"   • Confidence Score: {response.confidence_score:.2f}")
            print(f"   • Pipeline: {' → '.join(response.processing_pipeline)}")
            print(f"   • Documents Used: {len(response.retrieved_documents)}")
            
            print(f"\n📚 **TOP SOURCES:**")
            for j, result in enumerate(response.retrieved_documents[:3], 1):
                doc = result.document
                print(f"   {j}. 📄 {doc.title} (Score: {result.score:.3f}, Domain: {doc.domain.value})")
            
            results.append({
                'test_case': test_case['name'],
                'success': True,
                'processing_time': response.processing_time,
                'confidence': response.confidence_score
            })
            
        except Exception as e:
            print(f"❌ ERROR: {str(e)}")
            results.append({
                'test_case': test_case['name'],
                'success': False,
                'error': str(e)
            })
        
        print(f"\n{'🔸' * 30} END TEST CASE {i} {'🔸' * 30}")
    
    total_time = time.time() - total_start_time
    
    # Generate test report
    print(f"\n" + "=" * 60)
    print("📊 MODULAR RAG TEST REPORT")
    print("=" * 60)
    
    successful_tests = [r for r in results if r['success']]
    
    print(f"\n🎯 **OVERALL PERFORMANCE:**")
    print(f"   • Tests Passed: {len(successful_tests)}/{len(results)} ({len(successful_tests)/len(results)*100:.1f}%)")
    print(f"   • Total Time: {total_time:.2f}s")
    print(f"   • Avg Time per Query: {total_time/len(results):.2f}s")
    
    if successful_tests:
        avg_confidence = sum(r['confidence'] for r in successful_tests) / len(successful_tests)
        avg_processing_time = sum(r['processing_time'] for r in successful_tests) / len(successful_tests)
        
        print(f"\n📈 **QUALITY METRICS:**")
        print(f"   • Average Confidence: {avg_confidence:.3f}")
        print(f"   • Average Processing Time: {avg_processing_time:.3f}s")
    
    # System status
    status = modular_rag.get_system_status()
    print(f"\n🏆 **SYSTEM STATUS:**")
    print(f"   • Total System Queries: {status['total_queries']}")
    print(f"   • System Success Rate: {status['success_rate']:.1f}%")
    print(f"   • Active Users: {status['memory_stats']['active_users']}")
    
    print(f"\n🧩 **MODULAR CAPABILITIES DEMONSTRATED:**")
    capabilities = [
        "✅ Intelligent query understanding and classification",
        "✅ Adaptive routing based on query complexity", 
        "✅ Multi-strategy retrieval (semantic, keyword, hybrid)",
        "✅ Context-aware answer generation",
        "✅ User session and memory management",
        "✅ Comprehensive performance monitoring",
        "✅ Modular architecture with independent components"
    ]
    
    for capability in capabilities:
        print(f"   {capability}")
    
    return results

# Run the test suite
test_results = run_test_suite()


🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪 MODULAR RAG TEST SUITE 🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪

🔬 TEST CASE 1: Simple Factual Query
❓ Query: 'What is cloud computing?'

🔍 Processing: 'What is cloud computing?'
📝 Step 1: Query processing...
   Type: factual, Domain: technology
🚦 Step 2: Intelligent routing...
   Complexity: simple
   Strategy: semantic
   Est. time: 1.0s
🔍 Step 3: Document retrieval...
   Retrieved 3 documents
   1. Cloud Computing Fundamentals (Score: 0.754)
   2. Artificial Intelligence and Machine Learning (Score: 0.325)
   3. Quantum Computing Principles (Score: 0.277)
🤖 Step 4: Answer generation...
   Generated answer (Confidence: 0.90)
💾 Step 5: Memory update...

✅ Query processed in 2.56s

🎯 **FINAL ANSWER:**
   Cloud computing delivers computing services—including servers, storage, databases, networking, software, analytics, and intelligence—over the internet.  The main service models are Infrastructure as a Service (IaaS), Platform as a Service (PaaS), and Software as a Service (SaaS).  Ma

## 🎮 Interactive Demo

Try the Modular RAG system interactively:

In [ ]:
def interactive_demo():
    print("\n" + "🎮" * 20 + " MODULAR RAG INTERACTIVE DEMO " + "🎮" * 20)
    print("🧩 **MODULAR AI ASSISTANT**")
    print("🎮" * 60)
    print("Experience modular architecture! Each query is processed through")
    print("specialized modules that adapt to your question's complexity.")
    print("\nType 'quit' to exit, 'status' for system info, 'help' for commands")
    print("-" * 60)
    
    demo_user = "demo_user"
    query_count = 0
    
    example_queries = {
        "Simple": [
            "What is AI?",
            "Define cloud computing",
            "What is telemedicine?"
        ],
        "Comparative": [
            "Compare cloud computing with traditional computing",
            "Difference between AI and machine learning"
        ],
        "Complex": [
            "Analyze the impact of quantum computing on healthcare",
            "How does digital transformation affect business strategy?"
        ]
    }
    
    while True:
        try:
            user_input = input("\n🎯 Your question: ").strip()
            
            if user_input.lower() in ['quit', 'exit', 'q']:
                print("\n👋 Thank you for exploring Modular RAG!")
                break
            elif user_input.lower() == 'status':
                status = modular_rag.get_system_status()
                print(f"\n📊 **SYSTEM STATUS:**")
                print(f"   • Total Queries: {status['total_queries']}")
                print(f"   • Success Rate: {status['success_rate']:.1f}%")
                print(f"   • Avg Processing Time: {status['avg_processing_time']:.2f}s")
                print(f"   • Active Users: {status['memory_stats']['active_users']}")
                continue
            elif user_input.lower() == 'help':
                print(f"\n💡 **EXAMPLE QUERIES:**")
                for category, queries in example_queries.items():
                    print(f"\n🔸 **{category}:**")
                    for query in queries:
                        print(f"   • {query}")
                continue
            elif not user_input:
                print("Please enter a question or command.")
                continue
            
            query_count += 1
            print(f"\n🤔 Processing query #{query_count} with Modular RAG...")
            
            response = modular_rag.process_query(user_input, demo_user)
            
            print(f"\n🤖 **ANSWER:**")
            print(f"   {response.generated_answer}")
            
            print(f"\n📊 **ANALYSIS:**")
            print(f"   • Query Type: {response.query.query_type.value}")
            print(f"   • Domain: {response.query.domain.value}")
            print(f"   • Confidence: {response.confidence_score:.2f}")
            print(f"   • Processing Time: {response.processing_time:.2f}s")
            print(f"   • Pipeline: {' → '.join(response.processing_pipeline)}")
            
            if response.retrieved_documents:
                print(f"\n📚 **TOP SOURCES:**")
                for i, result in enumerate(response.retrieved_documents[:2], 1):
                    doc = result.document
                    print(f"   {i}. {doc.title} (Score: {result.score:.2f})")
        
        except KeyboardInterrupt:
            print("\n\n👋 Demo interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"\n❌ Error: {str(e)}")
    
    if query_count > 0:
        print(f"\n📈 **SESSION SUMMARY:**")
        print(f"   • Queries Processed: {query_count}")
        print(f"   • User Profile Created: ✅")
        print(f"   • Memory Updated: ✅")

print("\n💡 **To start interactive demo, uncomment and run the next cell**")
print("🎮 Experience real-time modular processing!")

In [ ]:
# Uncomment the line below to start interactive demo
# interactive_demo()

## 🎯 Summary & Next Steps

Congratulations! You've built a complete Modular RAG system.

In [1]:
def print_final_summary():
    print("\n" + "🎉" * 25 + " MISSION ACCOMPLISHED " + "🎉" * 25)
    print("\n🏆 **MODULAR RAG SYSTEM SUCCESSFULLY IMPLEMENTED!**")
    print("=" * 70)
    
    print("\n🧩 **MODULAR COMPONENTS BUILT:**")
    components = [
        "🔍 **QueryModule** - Intelligent query understanding and classification",
        "🔎 **RetrievalModule** - Multi-strategy document retrieval (semantic, keyword, hybrid)",
        "🤖 **GenerationModule** - Adaptive answer generation with Gemini integration",
        "💾 **MemoryModule** - User session and interaction history management",
        "🚦 **RouterModule** - Intelligent pipeline orchestration and routing",
        "🧩 **ModularRAGSystem** - Complete integration with performance monitoring"
    ]
    
    for component in components:
        print(f"   {component}")
    
    print("\n🚀 **KEY INNOVATIONS:**")
    innovations = [
        "🎯 **Adaptive Processing** - Pipeline adapts to query complexity",
        "🔧 **Pluggable Architecture** - Easy to swap and extend components",
        "📊 **Comprehensive Monitoring** - Performance tracking for each module",
        "🧠 **Context Awareness** - User preferences and conversation memory",
        "⚡ **Intelligent Routing** - Optimal strategy selection for each query",
        "🌍 **Multi-Domain Support** - Specialized handling for different domains"
    ]
    
    for innovation in innovations:
        print(f"   {innovation}")
    
    print("\n📈 **ADVANTAGES OVER BASIC RAG:**")
    advantages = [
        "🎯 **Higher Quality** - Specialized modules for different scenarios",
        "⚡ **Better Performance** - Adaptive processing pipelines",
        "🔧 **Easier Maintenance** - Independent module updates",
        "📊 **Better Observability** - Detailed monitoring and debugging",
        "🔄 **Greater Flexibility** - Mix and match components",
        "🏗️ **Scalability** - Independent scaling of modules"
    ]
    
    for advantage in advantages:
        print(f"   {advantage}")
    
    print("\n🔮 **NEXT LEVEL ENHANCEMENTS:**")
    enhancements = [
        "🤖 **Agentic Modules** - Add autonomous reasoning capabilities",
        "🔄 **Self-Improving System** - Modules that learn and adapt",
        "🌐 **Multi-Modal Support** - Text, image, audio integration",
        "📊 **GraphRAG Integration** - Knowledge graph reasoning",
        "🛡️ **Advanced Safety** - Content filtering and bias detection",
        "🏭 **Production Deployment** - Kubernetes and cloud scaling"
    ]
    
    for enhancement in enhancements:
        print(f"   {enhancement}")
    
    # Final system status
    status = modular_rag.get_system_status()
    print(f"\n📊 **FINAL SYSTEM STATUS:**")
    print(f"   • Modules Active: {len(modular_rag.modules)}/5")
    print(f"   • Total Queries Processed: {status['total_queries']}")
    print(f"   • System Success Rate: {status['success_rate']:.1f}%")
    print(f"   • Avg Processing Time: {status['avg_processing_time']:.2f}s")
    print(f"   • Users Tracked: {status['memory_stats']['active_users']}")
    print(f"   • Ready for Production: ✅")
    
    print("\n🎯 **READY FOR REAL-WORLD DEPLOYMENT!**")
    print("   Your Modular RAG system demonstrates enterprise-grade architecture")
    print("   and is ready to handle complex AI applications in production.")
    
    print("\n" + "🎉" * 70)
    print("   **Thank you for mastering Modular RAG architecture!**")
    print("🎉" * 70)

# Generate final summary
print_final_summary()

print(f"\n🔧 **QUICK START COMMANDS:**")
print(f"   • Test query: modular_rag.process_query('What is AI?')")
print(f"   • System status: modular_rag.get_system_status()")
print(f"   • Interactive demo: interactive_demo()")
print(f"   • Add new documents: retrieval_module.index_documents(new_docs)")


🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉 MISSION ACCOMPLISHED 🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉

🏆 **MODULAR RAG SYSTEM SUCCESSFULLY IMPLEMENTED!**

🧩 **MODULAR COMPONENTS BUILT:**
   🔍 **QueryModule** - Intelligent query understanding and classification
   🔎 **RetrievalModule** - Multi-strategy document retrieval (semantic, keyword, hybrid)
   🤖 **GenerationModule** - Adaptive answer generation with Gemini integration
   💾 **MemoryModule** - User session and interaction history management
   🚦 **RouterModule** - Intelligent pipeline orchestration and routing
   🧩 **ModularRAGSystem** - Complete integration with performance monitoring

🚀 **KEY INNOVATIONS:**
   🎯 **Adaptive Processing** - Pipeline adapts to query complexity
   🔧 **Pluggable Architecture** - Easy to swap and extend components
   📊 **Comprehensive Monitoring** - Performance tracking for each module
   🧠 **Context Awareness** - User preferences and conversation memory
   ⚡ **Intelligent Routing** - Optimal strategy selection for each query
   

NameError: name 'modular_rag' is not defined

## 📚 Resources & References

**Research Papers:**
- [Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks](https://arxiv.org/abs/2005.11401)
- [Self-RAG: Learning to Critique and Revise](https://arxiv.org/abs/2310.11511)
- [Modular Deep Learning](https://arxiv.org/abs/2302.11529)

**Implementation Frameworks:**
- [LangChain](https://python.langchain.com/) - LLM application framework
- [LlamaIndex](https://docs.llamaindex.ai/) - Data framework for LLMs
- [Haystack](https://haystack.deepset.ai/) - End-to-end NLP framework

**Vector Databases:**
- [Pinecone](https://www.pinecone.io/) - Managed vector database
- [Weaviate](https://weaviate.io/) - Open-source vector search
- [Qdrant](https://qdrant.tech/) - High-performance vector database

---

**🎯 Your Modular RAG system is now complete and ready for production!**

**Next Steps:**
1. 🚀 Deploy to cloud infrastructure
2. 🔬 Experiment with advanced techniques
3. 🎯 Customize for your specific domain
4. 📊 Scale with real-world data

**Happy Building! 🧩**